In [ ]:
pip install pandas numpy matplotlib gym torch tensorboardX ptan black flake8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gymnasium[atari] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of gymnasium[classic-control] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleFFDQN(nn.Module):
    def __init__(self, obs_len, actions_n):
        """
        Initialize the Simple Feedforward Dueling Deep Q-Network.

        Parameters:
        obs_len (int): Length of the input observation vector.
        actions_n (int): Number of possible actions.

        """
        super(SimpleFFDQN, self).__init__()

        # Value stream network
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

        # Advantage stream network
        self.fc_adv = nn.Sequential(
            nn.Linear(obs_len, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, actions_n)
        )

    def forward(self, x):
        """
        Forward pass through the network.

        Parameters:
        x (torch.Tensor): Input tensor.

        Returns:
        torch.Tensor: Output tensor containing the estimated Q-values.

        """
        # Compute value and advantage streams
        val = self.fc_val(x)
        adv = self.fc_adv(x)

        # Combine value and advantage streams to produce the final Q-values
        # The advantage values are centered by subtracting their mean to improve stability
        return val + (adv - adv.mean(dim=1, keepdim=True))

In [ ]:
import pandas as pd
import os
import csv
import glob
import numpy as np
import collections
import matplotlib.pyplot as plt

Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])
def bar2rel(df,tolerance):
    prev_vals = None
    fix_open_price  = True
    open, high, low, close, volume = [], [], [], [], []
    count_out = 0
    count_filter = 0
    count_fixed = 0
    for row in df.itertuples():
        val = (row._3,row._4,row._5,row._6,row._7)
        po, ph, pl,pc,pv = val
        if fix_open_price and prev_vals is not None:
            ppo, pph, ppl, ppc, ppv = prev_vals
            if abs(po - ppc) > 1e-8:
                count_fixed += 1
                po = ppc
                pl = min(pl, po)
                ph = max(ph, po)
                count_out += 1
        open.append(po)
        close.append(pc)
        high.append(ph)
        low.append(pl)
        volume.append(pv)
        prev_vals = val
    prices=Prices(open=np.array(open, dtype=np.float32),
                  high=np.array(high, dtype=np.float32),
                  low=np.array(low, dtype=np.float32),
                  close=np.array(close, dtype=np.float32),
                  volume=np.array(volume, dtype=np.float32))
    return prices_to_relative(prices)

def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)

def preprocess(path):
    df = pd.read_csv(os.path.abspath(train_path))

    index = ['<OPEN>', "<HIGH>", "<LOW>","<CLOSE>","<VOL>"]
    df[index] = df[index].astype(float)
    df_normalized = (df - df.min()) / (df.max() - df.min())
    # Define the tolerance value
    tolerance = 1e-8

    # Apply the lambda function to check if each value is within the tolerance of the first value
    df_normalized.applymap(lambda v: abs(v - df_normalized.iloc[0]) < tolerance)
    return bar2rel(df_normalized,tolerance)

In [ ]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np


DEFAULT_BARS_COUNT = 10
DEFAULT_COMMISSION_PERC = 0.1

class Actions(enum.Enum):
    Skip = 0
    Buy = 1
    Close = 2

class State:
    def __init__(self, bars_count, commission_perc,
                 reset_on_close, reward_on_close=True,
                 volumes=True):
        """
        Initializes the State object.

        Parameters:
        - bars_count (int): Number of bars (time periods) in the state representation.
        - commission_perc (float): Commission percentage applied to each trade.
        - reset_on_close (bool): Whether to reset the environment when a position is closed.
        - reward_on_close (bool): Whether to reward the agent immediately when a position is closed.
        - volumes (bool): Whether to include volume information in the state representation.
        """
        assert isinstance(bars_count, int)
        assert bars_count > 0
        assert isinstance(commission_perc, float)
        assert commission_perc >= 0.0
        assert isinstance(reset_on_close, bool)
        assert isinstance(reward_on_close, bool)
        self.bars_count = bars_count
        self.commission_perc = commission_perc
        self.reset_on_close = reset_on_close
        self.reward_on_close = reward_on_close
        self.volumes = volumes

    def reset(self, prices, offset):
        """
        Resets the state with new prices and offset.

        Parameters:
        - prices (Prices): Named tuple containing open, high, low, close, and volume prices.
        - offset (int): Offset index for the starting position in the price data.
        """
        assert isinstance(prices, Prices)
        assert offset >= self.bars_count-1
        self.have_position = False
        self.open_price = 0.0
        self._prices = prices
        self._offset = offset

    @property
    def shape(self):
        """
        Returns the shape of the state representation.
        """
        # [h, l, c] * bars + position_flag + rel_profit
        if self.volumes:
            return 4 * self.bars_count + 1 + 1,
        else:
            return 3*self.bars_count + 1 + 1,

    def encode(self):
        """
        Converts the current state into a numpy array.
        """
        res = np.ndarray(shape=self.shape, dtype=np.float32)
        shift = 0
        for bar_idx in range(-self.bars_count+1, 1):
            ofs = self._offset + bar_idx

            res[shift] = self._prices.high[ofs]
            shift += 1
            res[shift] = self._prices.low[ofs]
            shift += 1
            res[shift] = self._prices.close[ofs]
            shift += 1
            if self.volumes:
                res[shift] = self._prices.volume[ofs]
                shift += 1
        res[shift] = float(self.have_position)
        shift += 1
        if not self.have_position:
            res[shift] = 0.0
        else:
            res[shift] = self._cur_close() / self.open_price - 1.0
        return res

    def _cur_close(self):
        """
        Calculates the real close price for the current bar.
        """
        open = self._prices.open[self._offset]
        rel_close = self._prices.close[self._offset]
        return open * (1.0 + rel_close)

    def step(self, action):
        """
        Performs one step in the environment.

        Parameters:
        - action (Actions): Action taken by the agent.

        Returns:
        - reward (float): Reward obtained from the action.
        - done (bool): Whether the episode is done.
        """
        assert isinstance(action, Actions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == Actions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc
        elif action == Actions.Close and self.have_position:
            reward -= self.commission_perc
            done |= self.reset_on_close
            if self.reward_on_close:
                reward += 100.0 * (close / self.open_price - 1.0)
            self.have_position = False
            self.open_price = 0.0

        self._offset += 1
        prev_close = close
        close = self._cur_close()
        done |= self._offset >= self._prices.close.shape[0]-1

        if self.have_position and not self.reward_on_close:
            reward += 100.0 * (close / prev_close - 1.0)

        return reward, done

In [ ]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np
class StocksEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, prices: Prices, bars_count=DEFAULT_BARS_COUNT,
                 commission=DEFAULT_COMMISSION_PERC,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=False):
        """
        Initializes the StocksEnv environment.

        Parameters:
        - prices (Prices): Named tuple containing open, high, low, close, and volume prices.
        - bars_count (int): Number of bars (time periods) in the state representation.
        - commission (float): Commission percentage applied to each trade.
        - reset_on_close (bool): Whether to reset the environment when a position is closed.
        - state_1d (bool): Whether to represent the state in 1D format.
        - random_ofs_on_reset (bool): Whether to reset the environment with a random offset.
        - reward_on_close (bool): Whether to reward the agent immediately when a position is closed.
        - volumes (bool): Whether to include volume information in the state representation.
        """
        self._prices = prices
        self._state = State(
            bars_count, commission, reset_on_close,
            reward_on_close=reward_on_close, volumes=volumes)
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf,
            shape=self._state.shape, dtype=np.float32)
        self.random_ofs_on_reset = random_ofs_on_reset

    def seed(self, seed=None):
        """
        Sets the random seed for reproducibility.
        """
        self.np_random, seed1 = seeding.np_random(seed)
        seed2 = seeding.hash_seed(seed1 + 1) % 2 ** 31
        return [seed1, seed2]

    def reset(self):
        """
        Resets the environment to its initial state.

        Returns:
        - observation (np.ndarray): Initial observation/state of the environment.
        """
        prices = self._prices
        bar_count = self._state.bars_count
        # Check if prices have enough data for the requested bar count
        if len(prices) < bar_count:
            # Handle the case where prices have insufficient data
            return np.zeros(self._state.shape, dtype=np.float32)


        self._instrument = self.np_random.choice(
            list(self._prices._fields))
        if self._instrument is "open":
            prices = self._prices.open
        if self._instrument is "close":
            prices = self

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-2a4b3666468d>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._instrument is "open":
<ipython-input-6-2a4b3666468d>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._instrument is "close":


In [ ]:
import pandas as pd
import os
import csv
import glob
import numpy as np
import collections
import matplotlib.pyplot as plt

Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])
def bar2rel(df,tolerance):
    prev_vals = None
    fix_open_price  = True
    open, high, low, close, volume = [], [], [], [], []
    count_out = 0
    count_filter = 0
    count_fixed = 0
    for row in df.itertuples():
        val = (row[3],row[4],row[5],row[2],row[1])
        po, ph, pl,pc,pv = val
        if fix_open_price and prev_vals is not None:
            ppo, pph, ppl, ppc, ppv = prev_vals
            if abs(po - ppc) > 1e-8:
                count_fixed += 1
                po = ppc
                pl = min(pl, po)
                ph = max(ph, po)
                count_out += 1
        open.append(po)
        close.append(pc)
        high.append(ph)
        low.append(pl)
        volume.append(pv)
        prev_vals = val
    prices=Prices(open=np.array(open, dtype=np.float32),
                  high=np.array(high, dtype=np.float32),
                  low=np.array(low, dtype=np.float32),
                  close=np.array(close, dtype=np.float32),
                  volume=np.array(volume, dtype=np.float32))
    return prices_to_relative(prices)

def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)

def preprocess(path):
    df = pd.read_csv(os.path.abspath(train_path))

    index = ["open", "high", "low","close","volume"]
    df[index] = df[index].astype(float)
    df_normalized = (df[index] - df[index].min()) / (df[index].max() - df[index].min())
    # Define the tolerance value
    tolerance = 1e-8

    # Apply the lambda function to check if each value is within the tolerance of the first value
    df_normalized.applymap(lambda v: abs(v - df_normalized.iloc[0]) < tolerance)
    return bar2rel(df_normalized,tolerance)

In [ ]:
train_path = "ULTRACEMCO.NS_stock_data.csv"
rp=preprocess(train_path)
env  = StocksEnv(rp, bars_count=10,
                 commission=0.1,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=True)
obs = env.reset()
print(f"Initial observation: {obs}")
action_idx = env.action_space.sample()

FileNotFoundError: [Errno 2] No such file or directory: '/content/ULTRACEMCO.NS_stock_data.csv'

In [ ]:
# import os

# dataset_path = "/root/.cache/kagglehub/datasets/nitirajkulkarni/ultracemco-ns-stock-performance/versions/1"

# # List files in the directory
# print(os.listdir(dataset_path))



In [ ]:
import torch

# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# Get GPU details
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


In [ ]:
!pip install numpy torch gym tensorboardX ptan ignite

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from typing import Iterable, Union
from datetime import datetime, timedelta
import ptan
import pathlib
import argparse
import gym.wrappers
import torch.optim as optim
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Accuracy, Loss
from tensorboardX import SummaryWriter

In [ ]:
def calc_loss(batch, net, tgt_net, gamma, device="cpu"):
    states, actions, rewards, dones, next_states = unpack_batch(batch)
    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    if dones is not None:
        done_mask = torch.BoolTensor(dones.astype(bool)).to(device)
    else:
        done_mask =torch.BoolTensor(np.array([0],dtype =np.uint8))


    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    # get action performed in next state . i will take max score
    next_state_actions = net(next_states_v).max(1)[1]
    next_state_values = tgt_net.target_model(next_states_v).gather(1, next_state_actions.unsqueeze(-1)).squeeze(-1)
    next_state_values[done_mask] = 0.0
    # The Bellman equation is used to compute the expected Q-values for the current state-action pairs.
    expected_state_action_values = next_state_values.detach() * gamma + rewards_v
    # calculate loss between state_action_values and expected_state_action_values
    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [ ]:
# Constants and parameters
BATCH_SIZE = 32
BARS_COUNT = 10
EPS_START = 1.0
EPS_FINAL = 0.1
EPS_STEPS = 1000000
GAMMA = 0.99
REPLAY_SIZE = 50000
REPLAY_INITIAL = 50000
REWARD_STEPS = 2
LEARNING_RATE = 0.0001
STATES_TO_EVALUATE = 1000

# Initialize Tensorboard writer
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='logs')

METRICS = ('episode_reward', 'episode_steps', 'order_profits', 'order_steps')


In [ ]:
class EpsilonTracker:
    def __init__(self, selector, eps_start, eps_final, eps_steps):
        """
        Initializes the epsilon tracker to decay epsilon over training steps.

        :param selector: The epsilon-greedy action selector.
        :param eps_start: Initial epsilon value (high exploration).
        :param eps_final: Final epsilon value (low exploration).
        :param eps_steps: Number of steps over which epsilon decays.
        """
        self.selector = selector
        self.eps_start = eps_start
        self.eps_final = eps_final
        self.eps_steps = eps_steps
        self.epsilon = eps_start  # Start with initial epsilon

    def frame(self, frame_idx):
        """
        Updates the epsilon value based on the current training step.

        :param frame_idx: The current iteration/frame index.
        """
        # Linearly interpolate epsilon value
        self.epsilon = max(self.eps_final, self.eps_start - (frame_idx / self.eps_steps) * (self.eps_start - self.eps_final))
        self.selector.epsilon = self.epsilon  # Update epsilon in the action selector


In [ ]:
# Define batch generator function
def batch_generator(buffer: ptan.experience.ExperienceReplayBuffer, initial: int, batch_size: int):
    buffer.populate(initial)  # Initially populate buffer
    while True:
        # Ensure buffer is large enough before sampling
        # Change: Only populate if the buffer is not full

        yield buffer.sample(batch_size)

# Define training batch function
def train_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = calc_loss(batch=batch, net=net, tgt_net=tgt_net, gamma=GAMMA ** REWARD_STEPS, device=device)
    loss_v.backward()
    optimizer.step()
    eps_tracker.frame(engine.state.iteration)
    if getattr(engine.state, "eval_states", None) is None:
        eval_states = buffer.sample(STATES_TO_EVALUATE)
        eval_states = [np.array(transition.state, copy=False) for transition in eval_states]
        engine.state.eval_states = np.array(eval_states, copy=False)
    writer.add_scalar("training/loss", loss_v, engine.state.epoch)
    return {"loss": loss_v.item(), "epsilon": selector.epsilon}

# Validation function
import tqdm
def validation_run(env, net, episodes=100, device="cpu", epsilon=0.02, commission=0.1):
    stats = {metric: [] for metric in METRICS}

    progress_bar = tqdm.tqdm(range(episodes), desc="Validation Progress", leave=True)

    for episode in range(episodes):
        obs = env.reset()
        while obs is None:
            obs = env.reset()
        print(f"Initial observation: {obs}")
        total_reward = 0.0
        position = None
        position_steps = None
        episode_steps = 0

        while True:
            obs_v = torch.tensor([obs]).to(device)
            out_v = net(obs_v)
            action_idx = out_v.max(dim=1)[1].item()
            if np.random.random() < epsilon:
                action_idx = env.action_space.sample()
            action = Actions(action_idx)
            close_price = env._state._cur_close()

            if action == Actions.Buy and position is None:
                position = close_price
                position_steps = 0
            elif action == Actions.Close and position is not None:
                profit = close_price - position - (close_price + position) * commission / 100
                profit = 100.0 * profit / position
                stats['order_profits'].append(profit)
                stats['order_steps'].append(position_steps)
                position = None
                position_steps = None

            obs, reward, done, _ = env.step(action_idx)
            total_reward += reward
            episode_steps += 1
            if position_steps is not None:
                position_steps += 1
            if done:
                if position is not None:
                    profit = close_price - position - (close_price + position) * commission / 100
                    profit = 100.0 * profit / position
                    stats['order_profits'].append(profit)
                    stats['order_steps'].append(position_steps)
                break

        stats['episode_reward'].append(total_reward)
        stats['episode_steps'].append(episode_steps)
        progress_bar.update(1)

    progress_bar.close()

    return {key: np.mean(vals) for key, vals in stats.items()}

In [ ]:
from sklearn.model_selection import train_test_split
import ptan
# Assuming rp is your dataset
# Split the data into training and validation sets while preserving the Prices namedtuple structure
train_indices, val_indices = train_test_split(range(len(rp.open)), test_size=0.20, random_state=42)

# Create Prices namedtuples for training and validation sets
tp = Prices(open=rp.open[train_indices],
            high=rp.high[train_indices],
            low=rp.low[train_indices],
            close=rp.close[train_indices],
            volume=rp.volume[train_indices])

vp = Prices(open=rp.open[val_indices],
            high=rp.high[val_indices],
            low=rp.low[val_indices],
            close=rp.close[val_indices],
            volume=rp.volume[val_indices])

# Creating environments
env = StocksEnv(tp, bars_count=10, commission=0.1, reset_on_close=True, state_1d=False, random_ofs_on_reset=True, reward_on_close=False, volumes=True)
env_val = StocksEnv(vp, bars_count=10, commission=0.1, reset_on_close=True, state_1d=False, random_ofs_on_reset=True, reward_on_close=False, volumes=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initializing neural network models
net = SimpleFFDQN(env.observation_space.shape[0], env.action_space.n).to(device)
tgt_net = ptan.agent.TargetNet(net)

# Initializing action selector and epsilon tracker
selector = ptan.actions.EpsilonGreedyActionSelector(EPS_START)
eps_tracker = EpsilonTracker(selector, EPS_START, EPS_FINAL, EPS_STEPS)

# Initializing DQN agent
agent = ptan.agent.DQNAgent(net, selector, device=device)

# Initializing experience source
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, GAMMA, steps_count=REWARD_STEPS)

# Initializing optimizer
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

# Initializing Engine for training
trainer = Engine(train_batch)

In [ ]:
# Ignite event handlers
@trainer.on(Events.COMPLETED | Events.EPOCH_COMPLETED(every=10))
def log_training_results(engine):
    if engine.state.epoch % 10 == 0:
        res = validation_run(env_val, net, episodes=100, device="cpu", epsilon=0.02, commission=0.1)
        for key, value in res.items():
            writer.add_scalar("Agent Metrics", key, value)

@trainer.on(Events.ITERATION_COMPLETED)
def log_something(engine):
    out_dict = engine.state.output
    for key, value in out_dict.items():
        if value is None:
            value = 0.0
        elif isinstance(value, torch.Tensor):
            value = value.item()
        writer.add_scalar(f"Iteration Metrics{engine.state.epoch}/{key}", value, engine.state.iteration)

# Checkpointing
checkpoint_handler = ModelCheckpoint(dirname='saved_models', filename_prefix='checkpoint', n_saved=2, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'model': net})


# Training
# from ptan.experience import ExperienceSourceFirstLast
# experience_source = ExperienceSourceFirstLast(env, agent, gamma=0.99)
buffer = ptan.experience.ExperienceReplayBuffer(experience_source, buffer_size=10000)

trainer.run(batch_generator(buffer, REPLAY_INITIAL, BATCH_SIZE), max_epochs=100)
writer.close()
torch.save(net.state_dict(), 'model_state_dict.pth')
res = validation_run(env_val, net, episodes=100, device="cpu", epsilon=0.02, commission=0.1)


print(res)

In [ ]:
# Checkpointing
checkpoint_handler = ModelCheckpoint(dirname='saved_models', filename_prefix='checkpoint', n_saved=2, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'model': net})
trainer.run(batch_generator(buffer, REPLAY_INITIAL, BATCH_SIZE),max_epochs=100)
writer.close()
torch.save(net.state_dict(), 'model_state_dict.pth')
res=validation_run(env_val, net, episodes=100, device="cpu", epsilon=0.02, comission=0.1)
print(res)

In [ ]:
pip install streamlit pandas numpy scikit-learn joblib
pip install torch torchvision  # for PyTorch
pip install tensorflow         # for TensorFlow


In [ ]:
import joblib

# Example: Save a trained model
joblib.dump(model, "model.pkl")


In [ ]:
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("model.pkl")

# Streamlit UI
st.title("ML Model Deployment with Streamlit")

# User input
feature1 = st.number_input("Enter Feature 1:")
feature2 = st.number_input("Enter Feature 2:")

# Make a prediction
if st.button("Predict"):
    input_data = np.array([[feature1, feature2]])
    prediction = model.predict(input_data)
    st.write(f"Prediction: {prediction[0]}")


In [ ]:
streamlit run app.py
